In [ ]:
!python train.py --device 'cpu' --epochs 20 --save_interval 1 --train_photo_path train_photo --dataset shinkai --batch 4 --init_epochs 4 --use_sn 

In [ ]:
!python script/video_to_images.py --video-path ./dataset/video/city.gif

In [61]:
!python script/edge_smooth.py --dataset shinkai --image-size 256

usage: edge_smooth.py [-h] [--data_dir DATA_DIR] [--dataset DATASET]
                      [--image_size IMAGE_SIZE]
edge_smooth.py: error: unrecognized arguments: --image-size 256


In [23]:
!python inference_image.py --checkpoint checkpoint/generator_shinkai_19.pth --src dataset/test/HR_photo/2.jpg --dest 1.png

Weight loaded, ready to predict
Anime image saved to 1.png


In [59]:
from utils.common import load_checkpoint,load_weight
from modeling.anime_ganv2 import Generator
from modeling.anime_ganv2 import Discriminator
import os
def load_checkpoint(model, checkpoint_dir, posfix=''):
    path = os.path.join(checkpoint_dir, f'{model.name}{posfix}.pth')
    return load_weight(model, path)
G = Generator('shinkai')
D = Discriminator(args).to(args.device)

start_e = load_checkpoint(G, 'checkpoint')
test = load_checkpoint(D, 'checkpoint')
start_e

17

In [31]:
test

10

In [1]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='shinkai')
parser.add_argument('--data-dir', type=str, default='dataset')
parser.add_argument('--train_photo_path', type=str, default='train_photo')
parser.add_argument('--epochs', type=int, default=100)
parser.add_argument('--init_epochs', type=int, default=5)
parser.add_argument('--batch-size', type=int, default=6)
parser.add_argument('--checkpoint_dir', type=str, default='checkpoint')
parser.add_argument('--save_image_dir', type=str, default='dataset/predict_photo')
parser.add_argument('--gan_loss', type=str, default='lsgan', help='lsgan / hinge / bce')
parser.add_argument('--device', type=str, default='cpu')
#stor_true为如果命令行有该参数，则该参数设置为True,否则设置为False
parser.add_argument('--use_sn', action='store_true')
parser.add_argument('--save_interval', type=int, default=5)
parser.add_argument('--debug_samples', type=int, default=0)
parser.add_argument('--lr_g', type=float, default=2e-4)
parser.add_argument('--lr_d', type=float, default=4e-4)
parser.add_argument('--init_lr', type=float, default=1e-3)
parser.add_argument('--wadvg', type=float, default=10.0, help='Adversarial loss weight for G')
parser.add_argument('--wadvd', type=float, default=10.0, help='Adversarial loss weight for D')
parser.add_argument('--wcon', type=float, default=1.5, help='Content loss weight')
parser.add_argument('--wgra', type=float, default=3.0, help='Gram loss weight')
parser.add_argument('--wcol', type=float, default=30.0, help='Color loss weight')
parser.add_argument('--d_layers', type=int, default=3, help='Discriminator conv layers')
parser.add_argument('--d_noise', action='store_true')
args = parser.parse_args(args=[])

In [4]:
import cv2

In [10]:
def resize_image(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    h, w = image.shape[:2]

    if width and height:
        return cv2.resize(image, divisible((width, height)),  interpolation=inter)

    if width is None and height is None:
        return cv2.resize(image, divisible((w, h)),  interpolation=inter)

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, divisible(dim), interpolation=inter)
def divisible(dim):
    '''
    Make width and height divisible by 32
    '''
    width, height = dim
    return width - (width % 32), height - (height % 32)

In [18]:
image = cv2.imread('dataset/test/HR_photo/1.jpg')[:,:,::-1]

In [19]:
resize_image(image).shape

(928, 1248, 3)

In [20]:
image.shape

(937, 1279, 3)

In [ ]:
!python pytorch-animegan/train.py --device 'cpu' --epochs 20 --save_interval 1 --checkpoint_dir pytorch-animegan/checkpoint --train_photo_path train_photo --data-dir pytorch-animegan/dataset --dataset shinkai --batch 4 --init_epochs 4 --use_sn 

In [ ]:
import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()


if __name__ == "__main__":
    # 文件夹路径
    packagePath = '/kaggle/working/pytorch-animegan/checkpoint'
    zipPath = '/kaggle/working/output.zip'
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")
    print(datetime.datetime.utcnow())